# Bot Skeleton

Set up and authenticate Twitter account

In [2]:
import tweepy

#Setup and authenticate Tweepy
CONSUMER_KEY = ""
CONSUMER_SECRET = ""
ACCESS_TOKEN = ""
ACCESS_TOKEN_SECRET = ""

auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
api = tweepy.API(auth)

me = api.me()
print me.screen_name

PresAdjectives


Create a function for sending a tweet

In [10]:
import random
import json
import time

# If debug_mode is True then the bot won't actually tweet. Set debug_mode = False for it to tweet. 
debug_mode = True

# Functions that allow the bot to tweet or reply to tweets
def tweet(status):
    print "JUST TWEETED: ", status
    # Only *actually* send the tweet on twitter if we're not in debug mode
    if debug_mode == False:
        api.update_status(status)

The next cell will actually run the bot.

In [9]:
import requests
import json

# The list of candidate names and Twitter handles. Twitter handles are used so that searching for the candidates will find the bot's tweets
candidates = [["Donald","Trump", "@realDonaldTrump"],["Hillary","Clinton", "@HillaryClinton"],["Bernie","Sanders", "@BernieSanders"],
              ["Ted","Cruz", "@TedCruz"],["John","Kasich", "@JohnKasich"]]
# Other terms to search the NYT website for
other_terms = ["Democrat","Republican"]

# Create the string for searching the New York Times website. Only use last names so we don't find articles about other people with the same first names
search_terms = [c[1] for c in candidates] + other_terms
articles = []

search_string = ' OR '.join(['headline:' + term for term in search_terms])

# Only look at articles from the most recent day
begin_date_string = time.strftime("%Y%m%d")

# Use the New York Times API to find all articles from the most recent day that mention the candidates
response = requests.get('http://api.nytimes.com/svc/search/v2/articlesearch.json?fq=%s&begin_date=%s&api-key=7a6bb7d7d596c658b07bb1ecf1dd485b:13:74696179' % (search_string, begin_date_string))
html=response.text
values = json.loads(html)

new_articles = values['response']['docs']
articles += new_articles

# Find additional pages of results
i = 2
while len(new_articles) > 0:
    response = requests.get('http://api.nytimes.com/svc/search/v2/articlesearch.json?fq=%s&begin_date=%s&page=%d&api-key=7a6bb7d7d596c658b07bb1ecf1dd485b:13:74696179' % (search_string, begin_date_string, i))
    html=response.text
    try:
        values = json.loads(html)
        
        new_articles = values['response']['docs']
        articles += new_articles
    except Exception as e:
        break
    i += 1
    
print "Found " + str(len(articles)) + " articles"

Found 17 articles


In [11]:
from bs4 import BeautifulSoup
import nltk
import re
import string

# Initialize the data structure for storing the adjectives found
adjectives = {}
for candidate in candidates:
    adjectives[candidate[1]] = {}

# Look through each article
for (index, article) in enumerate(articles):
    print index
    
    # Download the HTML of the article's webpage
    response = requests.get(article['web_url'])
    html = response.text

    # Find the body text of the article by joining the data in each div with class 'story-content'
    article_bs = BeautifulSoup(html)
    article_text = "\n".join([a.text for a in article_bs.find_all(class_="story-content")])
    
    # Split the text by sentence
    sentences = re.split('\.|\?|\,', article_text)
    
    # Look through the sentences
    for sentence in sentences:
        for candidate in candidates:
            # If a candidate is mentioned in the sentence
            if candidate[0] in sentence or candidate[1] in sentence:
                # Use a regular expression to remove punctuation on the outside of words and then split by word
                # This should keep punctuation in cases such as contractions (isn't), but not, for example quotes ("Hello")
                tokens = nltk.word_tokenize(re.sub('([^a-zA-Z0-9_ ]* [^a-zA-Z0-9_ ]*|^[^a-zA-Z0-9_ ]*|[^a-zA-Z0-9_ ]*$)+', ' ', sentence))
                
                # Use the Natural Language Toolkit to find the part of speech of each word in the sentence
                pairs = nltk.pos_tag(tokens)
                
                # Look through word by word
                for (i, pair) in enumerate(pairs):
                    # If the word is an adjective, test if it is either in the form <adjective> <candidate> (e.g. "great Hillary Clinton")
                    # or the form <candidate> "is"/"was" <adjective> (e.g. "Ted Cruz is great")
                    if pair[1] == "JJ" and ((i < len(pairs) - 1 and pairs[i+1][0] in candidate)
                                           or (i > 1 and pairs[i - 2][0] in candidate and pairs[i-1][0] in ["is", "was"])):
                        # If the adjective has already been found, increment the count of how many times it's been found
                        if pair[0].lower() in adjectives[candidate[1]]:
                            adjectives[candidate[1]][pair[0].lower()] += 1
                        # If it's new, add it to the candidate's list of adjectives
                        else:
                            adjectives[candidate[1]][pair[0].lower()] = 1
                            # If a candidate has three distinct adjectives, tweet them and remove those adjectives from the list
                            if len(adjectives[candidate[1]]) == 3:
                                tweet("%s is '%s', '%s', and '%s'" % (candidate[2], adjectives[candidate[1]].keys()[0],
                                                                        adjectives[candidate[1]].keys()[1], adjectives[candidate[1]].keys()[2]))

                                keys = adjectives[candidate[1]].keys()
                                for key in keys:
                                    del adjectives[candidate[1]][key]
                        print adjectives
                        print sentence
                        print pairs
                        print re.sub('([^a-zA-Z0-9_ ]* [^a-zA-Z0-9_ ]*|^[^a-zA-Z0-9_ ]*|[^a-zA-Z0-9_ ]*$)+', ' ', sentence)
print adjectives

0
{'Sanders': {}, 'Cruz': {}, 'Clinton': {}, 'Trump': {u'unique': 1}, 'Kasich': {}}
 Trump is unique
[(u'Trump', 'NNP'), (u'is', 'VBZ'), (u'unique', 'JJ')]
 Trump is unique 
1
2
3
{'Sanders': {}, 'Cruz': {}, 'Clinton': {}, 'Trump': {u'unique': 1, u'anti-semitic': 1}, 'Kasich': {}}
 Trump is anti-Semitic
[(u'Trump', 'NN'), (u'is', 'VBZ'), (u'anti-Semitic', 'JJ')]
 Trump is anti-Semitic 
4
5
{'Sanders': {}, 'Cruz': {}, 'Clinton': {u'rival': 1}, 'Trump': {u'unique': 1, u'anti-semitic': 1}, 'Kasich': {}}
 Democratic presidential candidate Bernie Sanders outraised rival Hillary Clinton in February but spent at a faster pace
[(u'Democratic', 'JJ'), (u'presidential', 'JJ'), (u'candidate', 'NN'), (u'Bernie', 'NNP'), (u'Sanders', 'NNP'), (u'outraised', 'VBD'), (u'rival', 'JJ'), (u'Hillary', 'NNP'), (u'Clinton', 'NNP'), (u'in', 'IN'), (u'February', 'NNP'), (u'but', 'CC'), (u'spent', 'VBN'), (u'at', 'IN'), (u'a', 'DT'), (u'faster', 'JJR'), (u'pace', 'NN')]
 Democratic presidential candidate Berni

C:\Anaconda2\lib\site-packages\bs4\__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))
